In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "jarvis"
#MODEL = "mixtral:8x7b"
#MODEL = "llama2"


In [2]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

# model.invoke("Tell me a joke")

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
# chain.invoke("Tell me a joke")

In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [5]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's my name'?"})

'\nYour name is Santiago. As Jarvis would confidently provide accurate information to assist with any query related to identity or personal details given the context provided and following its logical sequence without speculation, assuming such a direct question within his designed role as an artificial assistant system could not be answered due to insufficient data outside of this limited scripted interaction.\n\n'

In [7]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = os.path.abspath('../sample.pdf')
print(f"Absolute path to PDF: {pdf_path}")

loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()
pages

Absolute path to PDF: /app/sample.pdf


[Document(metadata={'source': '/app/sample.pdf', 'page': 0}, page_content='Generative AI Solutions\nStep outside the hype cycle with generative AI built with business in mind. Altair® RapidMiner®’s\ngenerative AI tool puts access to hundreds of thousands of large language models (LLMs) at\nusers’ ﬁngertips. Our versatile and customizable extension oﬀers a secure, cost-eﬀective, and\neasy-to-use way to leverage generative AI’s capabilities.\nSee for yourself with our explainer: Three Clicks to Generative AI.\nWatch Video'),
 Document(metadata={'source': '/app/sample.pdf', 'page': 1}, page_content='More Accurate Results with\nFewer Hallucinations\nThe Altair RapidMiner extension supports retrieval-\naugmented generation (RAG), a below-the-surface\nprocess that limits the model’s frame of reference to real\nand vetted information. RAG adds context to queries that\nincrease the accuracy of the result and reduces the\nlikelihood of AI “hallucinating.” Get results you can trust\nwith our gen

In [8]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [9]:
retriever = vectorstore.as_retriever()
retriever.invoke(" Generative AI")

[Document(metadata={'source': '/app/sample.pdf', 'page': 3}, page_content="A Human's Guide to Generative AI\nIn this eGuide, we’ll provide a levelheaded, easy-\nto-understand overview of generative AI\ntechnology – what it is, where it came from, how\nit works, and what it can (and can’t) do. We’ll go\nover the considerations and responsibilities of…\neGuide\nPanel Discussion: Changing the Pace of\nManufacturing Transformation through…\nWith the emergence of ChatGPT we have reached\nanother pivotal moment with AI. At the very least,\nit has unlocked a next-level fever of interest in AI\nand potentially the ability to disrupt whole\nindustries. This has arguably most organizationa…\nATCx AI for Engineers\nAccelerate AI Adoption with Frictionless AI\n Three Clicks to Generative AI"),
 Document(metadata={'source': '/app/sample.pdf', 'page': 1}, page_content='More Accurate Results with\nFewer Hallucinations\nThe Altair RapidMiner extension supports retrieval-\naugmented generation (RAG), a

In [10]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [11]:
questions = [
    "What is generative AI?",
    "Does this support RAG",
    "Are their pre-trained models",
    "Can we trust RAG results",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is generative AI?
Answer: 
Generative AI refers to a subset of artificial intelligence technologies, specifically focused on the generation of new content based on data input. It's often used in natural language processing tasks such as text completion and augmentation where it can produce diverse types of content like articles or social media posts tailored for specific purposes.

Question: Describe human guide to generative ai
Answer: In this eGuide, we aim to provide a comprehensive yet easily understandable overview of the technologies surrounding generative artificial intelligence (generative AI). We'll start by defining what generative AI technology is and diving into its origins. Moving forward, it’s crucial for users to know how this powerful tool functions - explaining the workings of machine learning in layman terms makes understanding possible even without technical expertise.

We won't stop at just exploring what generative AI can do; we will also tackle its 